# Multi-task recommenders


In the [Movielens retrieval tutorial](https://github.com/tensorflow/recommenders/examples/tfrs_movielens.ipynb) we built a retrieval system using movie watches as positive interaction signals.

In many applications, however, there are multiple rich sources of feedback to draw upon. For example, an e-commerce site may record user visits to product pages (abundant, but relatively low signal), image clicks, adding to cart, and, finally, purchases. It may even record post-purchase signals such as reviews and returns.

Integrating all these different forms of feedback is critical to building systems that users love to use, and that do not optimize for any one metric at the expense of overall performance.

In this tutorial, we are going to build a multi-objective recommender for Movielens, using both implicit (movie watches) and explicit signals (ratings).

## Imports


Let's first get our imports out of the way.


In [0]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as  tf

In [0]:
import tensorflow_recommenders as tfrs

## Preparing the dataset

As before, we're going to use the Movielens 100K dataset.

In [0]:
ratings, movies = tfrs.datasets.movielens_100K()

And repeat our preparations for building vocabularies and splitting the data into a train and a test set:

In [0]:
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

movie_ids = ratings.batch(100_000).map(lambda x: x["movie_id"])
user_ids = ratings.batch(100_000).map(lambda x: x["user_id"])

unique_movie_ids = np.unique(np.concatenate(list(movie_ids)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

## A multi-task model

There are two critical parts to multi-task recommenders:

1. They optimize for two or more objectives, and so have two or more losses.
2. They share variables between the tasks, allowing for transfer learning.

In this tutorial, we will define our models as before, but instead of having  a single task, we will have two tasks: one that predicts ratings, and one that predicts movie watches.

The user and movie models are as before:

In [0]:
embedding_dimension = 32

# We wrap these in functions this time so that we can run multiple independent
# models later.
def user_model():
  user_features = [tf.feature_column.embedding_column(
          tf.feature_column.categorical_column_with_vocabulary_list(
              "user_id", unique_user_ids),
          embedding_dimension)]
  return tf.keras.Sequential([tf.keras.layers.DenseFeatures(user_features)])


def movie_model():
  movie_features = [tf.feature_column.embedding_column(
    tf.feature_column.categorical_column_with_vocabulary_list(
      "movie_id", list(unique_movie_ids)),
    embedding_dimension)]
  return  tf.keras.Sequential([tf.keras.layers.DenseFeatures(movie_features)])

However, now we will have two tasks. The first is the rating task:

In [0]:
tfrs.tasks.RankingTask(
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanAbsoluteError()]
)

Its goal is to predict the ratings as accurately as possible.

The second is the retrieval task:

In [0]:
tfrs.tasks.RetrievalTask(
  corpus_metrics=tfrs.metrics.FactorizedTopK(
    candidates=movies.batch(128)
  )
)

As before, this task's goal is to predict which movies the user will or will not watch.

### Putting it together

We put it all together in a model class.

The new component here is that - since we have two tasks and two losses - we need to decide on how important each loss is. We can do this by giving each of the losses a weight, and treating these weights as hyperparameters. If we assign a large loss weight to the rating task, our model is going to focus on predicting ratings (but still use some information from the retrieval task); if we assign a large loss weight to the retrieval task, it will focus on retrieval instead.

In [0]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    # Embeddings as before.
    self.movie_model: tf.keras.layers.Layer = movie_model()
    self.user_model: tf.keras.layers.Layer = user_model()

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(128, activation="relu"),
      tf.keras.layers.Dense(1)
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.RankingTask(
      loss=tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.MeanAbsoluteError()]
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.RetrievalTask(
      corpus_metrics=tfrs.metrics.FactorizedTopK(
        candidates=movies.batch(128).map(self.movie_model)
      )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model({"user_id": features["user_id"]})
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    movie_embeddings = self.movie_model(
        {"movie_id": features["movie_id"]})
    
    return (
        user_embeddings,
        movie_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, movie_embeddings], axis=1))
    )

  def train_loss(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    user_embeddings, movie_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(labels=features["rating"],
                                   predictions=rating_predictions)
    retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

### Rating-specialized model

Depending on the weights we assign, the model will encode a different balance of the tasks. Let's start with a model that only considers ratings.

In [0]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=0.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [0]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [0]:
model.fit(cached_train, epochs=10)
model.evaluate(cached_test, return_dict=True)

Epoch 1/10
10/10 [==============================] - 3s 336ms/step - mean_absolute_error: 2.4425 - factorized_top_k_1: 0.0194 - factorized_top_k_1/top_1_categorical_accuracy: 4.3750e-04 - factorized_top_k_1/top_5_categorical_accuracy: 0.0027 - factorized_top_k_1/top_10_categorical_accuracy: 0.0055 - factorized_top_k_1/top_50_categorical_accuracy: 0.0295 - factorized_top_k_1/top_100_categorical_accuracy: 0.0585 - loss: 7.3609
Epoch 2/10
10/10 [==============================] - 3s 321ms/step - mean_absolute_error: 0.9370 - factorized_top_k_1: 0.0194 - factorized_top_k_1/top_1_categorical_accuracy: 4.5000e-04 - factorized_top_k_1/top_5_categorical_accuracy: 0.0027 - factorized_top_k_1/top_10_categorical_accuracy: 0.0056 - factorized_top_k_1/top_50_categorical_accuracy: 0.0296 - factorized_top_k_1/top_100_categorical_accuracy: 0.0585 - loss: 1.2698
Epoch 3/10
10/10 [==============================] - 3s 318ms/step - mean_absolute_error: 0.9273 - factorized_top_k_1: 0.0194 - factorized_top_k_

{'mean_absolute_error': 0.8905865550041199,
 'factorized_top_k_1': array([0.0008 , 0.0037 , 0.00715, 0.03115, 0.06225], dtype=float32),
 'factorized_top_k_1/top_1_categorical_accuracy': 0.0007999999797903001,
 'factorized_top_k_1/top_5_categorical_accuracy': 0.003700000001117587,
 'factorized_top_k_1/top_10_categorical_accuracy': 0.007149999961256981,
 'factorized_top_k_1/top_50_categorical_accuracy': 0.031150000169873238,
 'factorized_top_k_1/top_100_categorical_accuracy': 0.06224999949336052,
 'loss': 1.2088614702224731}

The model does OK on predicting ratings (with an MSE of around 0.79), but performs poorly at predicting which movies will be watched or not: its accuracy at 100 is almost 3 times worse than a model trained solely to predict watches.

### Retrieval-specialized model

Let's now try a model that focuses on retrieval only.

In [0]:
model = MovielensModel(rating_weight=0.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [0]:
model.fit(cached_train, epochs=10)
model.evaluate(cached_test, return_dict=True)

Epoch 1/10
10/10 [==============================] - 3s 317ms/step - mean_absolute_error: 3.5752 - factorized_top_k_2: 0.0269 - factorized_top_k_2/top_1_categorical_accuracy: 4.0000e-04 - factorized_top_k_2/top_5_categorical_accuracy: 0.0036 - factorized_top_k_2/top_10_categorical_accuracy: 0.0072 - factorized_top_k_2/top_50_categorical_accuracy: 0.0413 - factorized_top_k_2/top_100_categorical_accuracy: 0.0820 - loss: 70217.8551
Epoch 2/10
10/10 [==============================] - 3s 311ms/step - mean_absolute_error: 3.5976 - factorized_top_k_2: 0.1075 - factorized_top_k_2/top_1_categorical_accuracy: 0.0030 - factorized_top_k_2/top_5_categorical_accuracy: 0.0208 - factorized_top_k_2/top_10_categorical_accuracy: 0.0418 - factorized_top_k_2/top_50_categorical_accuracy: 0.1767 - factorized_top_k_2/top_100_categorical_accuracy: 0.2954 - loss: 67798.1534
Epoch 3/10
10/10 [==============================] - 3s 314ms/step - mean_absolute_error: 3.6113 - factorized_top_k_2: 0.1198 - factorized_to

{'mean_absolute_error': 3.642604351043701,
 'factorized_top_k_2': array([1.0000e-04, 2.5000e-03, 7.1000e-03, 7.5650e-02, 1.6575e-01],
       dtype=float32),
 'factorized_top_k_2/top_1_categorical_accuracy': 9.999999747378752e-05,
 'factorized_top_k_2/top_5_categorical_accuracy': 0.0024999999441206455,
 'factorized_top_k_2/top_10_categorical_accuracy': 0.0071000000461936,
 'factorized_top_k_2/top_50_categorical_accuracy': 0.07564999908208847,
 'factorized_top_k_2/top_100_categorical_accuracy': 0.16574999690055847,
 'loss': 29010.5}

We get the opposite result: a model that does well on retrieval, but poorly on predicting ratings.

### Equally-weighted model

Let's now train a model that weighs both tasks equally.

In [0]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [0]:
model.fit(cached_train, epochs=10)
model.evaluate(cached_test, return_dict=True)

Epoch 1/10
10/10 [==============================] - 3s 319ms/step - mean_absolute_error: 2.3338 - factorized_top_k_3: 0.0263 - factorized_top_k_3/top_1_categorical_accuracy: 3.1250e-04 - factorized_top_k_3/top_5_categorical_accuracy: 0.0031 - factorized_top_k_3/top_10_categorical_accuracy: 0.0070 - factorized_top_k_3/top_50_categorical_accuracy: 0.0402 - factorized_top_k_3/top_100_categorical_accuracy: 0.0807 - loss: 70222.5533
Epoch 2/10
10/10 [==============================] - 3s 316ms/step - mean_absolute_error: 0.9475 - factorized_top_k_3: 0.1010 - factorized_top_k_3/top_1_categorical_accuracy: 0.0027 - factorized_top_k_3/top_5_categorical_accuracy: 0.0191 - factorized_top_k_3/top_10_categorical_accuracy: 0.0388 - factorized_top_k_3/top_50_categorical_accuracy: 0.1652 - factorized_top_k_3/top_100_categorical_accuracy: 0.2793 - loss: 67918.7834
Epoch 3/10
10/10 [==============================] - 3s 317ms/step - mean_absolute_error: 0.9153 - factorized_top_k_3: 0.1188 - factorized_to

{'mean_absolute_error': 0.815298855304718,
 'factorized_top_k_3': array([0.0002 , 0.00225, 0.0073 , 0.07455, 0.1693 ], dtype=float32),
 'factorized_top_k_3/top_1_categorical_accuracy': 0.00019999999494757503,
 'factorized_top_k_3/top_5_categorical_accuracy': 0.0022499999031424522,
 'factorized_top_k_3/top_10_categorical_accuracy': 0.007300000172108412,
 'factorized_top_k_3/top_50_categorical_accuracy': 0.07455000281333923,
 'factorized_top_k_3/top_100_categorical_accuracy': 0.16930000483989716,
 'loss': 28973.927734375}

The result is a model that does better on both tasks. This is the promise of transfer learning, where jointly training a model on a set of related tasks can result in a better model that training a set of separate estimators.

Of course, this is not true everywhere. We can expect better results when tasks are closely related, or when we can trasnfer knowledge from a data-abundant task (such as clicks) to a related data-sparse task (such as purchases).